In [1]:
data_var = '2024-02-14'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12028,2024-02-14,Brasil Nbb,19:00,Corinthians Paulista,Franca,3.02,1.35,161.5,1.86,1.84,5.5,2.00,1.72,Uwi4PMD9,0.331126,0.740741,0.537634,0.543478,0.071867,0.4,0.6,NaN,NaN,123.342,40.951386,0.332015,123.792,29.733034,0.240185,133.658,130.214,119.00,97.58,0.0,0.0,0.0,0.0,0.540443,0.007644,0.106446,-2.64,-0.528,-3.825758,0.647360,0.4,-0.247360,0.67,0.134,2.611940,0.686514,0.8,0.113486
12029,2024-02-14,Brasil Nbb,19:30,São José,União Corinthians,1.22,3.98,156.5,1.85,1.85,-10.5,2.00,1.72,2ikCN0rM,0.819672,0.251256,0.540541,0.540541,0.070928,0.4,0.6,NaN,NaN,150.304,51.004420,0.339342,276.860,235.483818,0.850552,111.854,357.468,92.30,670.95,0.0,1.0,0.0,0.0,0.750621,0.000000,0.106446,-2.42,-0.484,-0.454545,0.606771,0.6,-0.006771,-2.47,-0.494,-6.032389,0.261901,0.3,0.038099
12030,2024-02-14,Brasil Nbb,20:00,Mogi,Caxias do Sul,1.65,2.15,152.5,1.86,1.80,-3.5,2.00,1.72,0x8FMKcS,0.606061,0.465116,0.537634,0.555556,0.071177,0.4,0.8,NaN,NaN,282.422,153.982496,0.545221,213.000,122.397150,0.574635,284.612,240.620,171.50,397.42,0.0,0.0,0.0,0.0,0.186081,0.023184,0.106446,-5.00,-1.000,-0.650000,0.383176,0.1,-0.283176,-2.80,-0.560,-2.053571,0.278133,0.2,-0.078133
12031,2024-02-14,Brasil Nbb,20:30,Fortaleza B.C.,Botafogo,1.22,3.98,161.5,1.87,1.79,-10.5,2.00,1.72,8Uk8OtTF,0.819672,0.251256,0.534759,0.558659,0.070928,0.6,0.8,NaN,NaN,174.260,68.877958,0.395260,282.940,183.292899,0.647815,144.666,334.252,86.70,371.52,0.0,0.0,0.0,0.0,0.750621,0.030912,0.106446,-1.78,-0.356,-0.617978,0.674816,0.5,-0.174816,2.49,0.498,5.983936,0.246416,0.2,-0.046416
12032,2024-02-14,Eua Nba,00:00,Phoenix Suns,Sacramento Kings,1.53,2.61,244.5,1.88,2.04,-4.5,1.97,1.97,42ExbfVM,0.653595,0.383142,0.531915,0.490196,0.036737,0.4,0.6,NaN,NaN,180.928,20.131789,0.111270,244.396,69.461301,0.284216,176.064,237.392,167.70,265.55,0.0,0.0,0.0,0.0,0.368925,0.057723,0.000000,2.43,0.486,1.090535,0.708646,0.8,0.091354,-1.59,-0.318,-5.062893,0.503960,0.4,-0.103960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12162,2024-02-14,Portugal Taça Da Liga,17:00,Sporting CP,FC Porto,1.55,2.34,162.5,1.82,1.88,-5.5,2.00,1.72,rF0mhz3Q,0.645161,0.427350,0.549451,0.531915,0.072512,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,103.02,0.0,0.0,0.0,0.0,0.287205,0.022933,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12163,2024-02-14,Portugal Taça Da Liga,17:30,Vitória SC,Imortal,1.73,2.01,158.5,1.87,1.83,-3.5,2.07,1.67,pOnFmfBs,0.578035,0.497512,0.534759,0.546448,0.075547,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.105877,0.015289,0.151253,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12164,2024-02-14,Portugal Taça Da Liga,18:00,Benfica,Esgueira,1.01,28.00,157.5,1.80,1.90,-28.5,1.85,1.85,6Tk9vHYQ,0.990099,0.035714,0.555556,0.526316,0.025813,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,303.88,0.0,0.0,0.0,0.0,1.315740,0.038222,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12165,2024-02-14,Portugal Taça Da Liga,18:00,Oliveirense,CD Póvoa,1.18,4.42,158.5,1.80,1.90,-10.5,1.85,1.85,hQo5ucJK,0.847458,0.226244,0.555556,0.526316,0.073702,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,138.75,167.67,0.0,0.0,0.0,0.0,0.8182

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Phoenix Suns,Sacramento Kings,244.5,1.88,1.0000,Over
1,00:00,Eua Nba,Portland Trail Blazers,Minnesota Timberwolves,211.5,1.85,1.0000,Over
2,00:30,Eua Nba,Los Angeles Lakers,Detroit Pistons,241.5,1.85,1.0000,Over
3,21:00,Eua Nba,Charlotte Hornets,Atlanta Hawks,238.5,1.89,1.0000,Over
4,21:00,Eua Nba,Orlando Magic,New York Knicks,213.5,1.85,1.0000,Over
5,21:00,Eua Nba,Philadelphia 76ers,Miami Heat,222.5,1.85,1.0000,Over
6,21:30,Eua Nba,Boston Celtics,Brooklyn Nets,225.5,1.90,1.0000,Over
7,21:30,Eua Nba,Cleveland Cavaliers,Chicago Bulls,220.5,1.86,1.0000,Over
8,22:00,Eua Nba,Memphis Grizzlies,Houston Rockets,216.5,1.82,1.0000,Over
9,22:00,Eua Nba,New Orleans Pelicans,Washington Wizards,234.5,1.82,0.9999,Over
